# config

> This module provides the core config class for qgofer

In [9]:
#| default_exp config

In [1]:
%pip install black

Note: you may need to restart the kernel to use updated packages.


In [2]:
#| hide
from nbdev.showdoc import *

In [4]:
#| hide
%run 00_core.ipynb
%run 02_logger.ipynb
import sys
sys.path.append("..")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
#| export
from __future__ import annotations
import shutil
from typing import Dict, Any
from pathlib import Path

In [6]:
#| export 
try:
    from qgoferutils.logger import get_log_path, get_logger
except:
    pass

In [12]:
#| export


log_path = get_log_path()
logger = get_logger(log_path=log_path)


class QGoferConfig:
    """The main appliation class for managing the entirety of qgofer.

    Attributes:
        home: The home directory of the user, defaults to Path.home()
        root_dir: The root directory to index, defaults to Path.home()
    """

    __slots__ = (
        "_home",
        "_root_dir",
        "_qgofer",
        "_qgofer_config",
        "_qgofer_cache",
        "_qgofer_cache_db",
        "_qgofer_logs",
    )
    _instances: Dict[Any, Any] = {}

    def __new__(cls, home, root_dir) -> QGoferConfig:
        """Create a new instance of the class. by using the singleton pattern."""
        if cls not in cls._instances:
            cls._instances[cls] = super(App, cls).__new__(cls)
        return cls._instances[cls]

    def __init__(self, home: Path = Path.home(), root_dir: Path = Path.home()):
        """Initialize the class. using home and root_dir as the home and root directory of qgofer."""
        self._home = home
        self._root_dir = root_dir
        self._qgofer = self._home / '.qgofer'
        self._qgofer.mkdir(parents=True, exist_ok=True)
        self._qgofer_config = self._qgofer / 'config.toml'
        self._qgofer_config.touch(exist_ok=True)
        self._qgofer_cache = self._qgofer / 'cache'
        self._qgofer_cache.mkdir(parents=True, exist_ok=True)
        self._qgofer_cache_db = self._qgofer_cache / 'qgofer.db'
        self._qgofer_cache_db.touch(exist_ok=True)
        self._qgofer_logs = self._qgofer / 'logs'
        self._qgofer_logs.mkdir(parents=True, exist_ok=True)

    @property
    def home(self) -> Path:
        """Return the home directory of the user."""
        return self._home

    @property
    def qgofer(self) -> Path:
        """Return the qgofer directory."""
        return self._qgofer

    @property
    def qgofer_config(self) -> Path:
        """Return the qgofer config file."""
        return self._qgofer_config

    @property
    def qgofer_cache(self) -> Path:
        """Return the qgofer cache directory."""
        return self._qgofer_cache

    @property
    def qgofer_cache_db(self) -> Path:
        """Return the qgofer cache database."""
        return self._qgofer_cache_db

    def _clear_cache_db(self) -> None:
        """Clear the qgofer cache database."""
        qgofer_db_wrapper = QGoferDBWrapper(self.qgofer_cache_db)
        try:
            os.remove(self.qgofer_cache_db)
        except FileNotFoundError as err:
            logger.error(err)

    def clear_cache(self) -> None:
        """Clears the cache folder"""
        try:
            shutil.rmtree(self._qgofer_cache)
        except OSError as err:
            logger.error(err)
        self._clear_cache_db()

    def clear_logs(self) -> None:
        """This function clears the logs folder
        """
        try:
            shutil.rmtree(self._qgofer_logs)
            self._qgofer_logs.mkdir(parents=True, exist_ok=True)
        except OSError as err:
            logger.error(err)

    def __repr__(self) -> str:
        """Return the representation of the class."""
        return f"{self.__class__.__name__}({self.home})"

    def __str__(self) -> str:
        """Return the string representation of the class."""
        return "Qgofer your personal assistant"

In [10]:
#| hide
import nbdev; nbdev.nbdev_export()